In [39]:
!pip -q install langchain openai==0.27.0 tiktoken

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


# Summarization

History  
Challenges  
Fine-tuning  
Instruct Tuning

In [40]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

In [41]:
!pip show langchain

Name: langchain
Version: 0.0.184
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\User\anaconda3\envs\new\Lib\site-packages
Requires: aiohttp, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Setting up Summarization Chain

In [42]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.5)

In [43]:
text_splitter = CharacterTextSplitter()

In [44]:
# load the doc
with open('paper3.txt') as f:
    how_to_win_friends = f.read()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=170, chunk_overlap=0
)
texts = text_splitter.split_text(how_to_win_friends)

Created a chunk of size 601, which is longer than the specified 170
Created a chunk of size 2064, which is longer than the specified 170
Created a chunk of size 1698, which is longer than the specified 170
Created a chunk of size 4963, which is longer than the specified 170
Created a chunk of size 2992, which is longer than the specified 170
Created a chunk of size 1462, which is longer than the specified 170
Created a chunk of size 3135, which is longer than the specified 170
Created a chunk of size 4666, which is longer than the specified 170
Created a chunk of size 3084, which is longer than the specified 170


In [45]:
texts[1]

'Abstract: We investigate the transformative potential of large language models\n                                (LLMs) in the legal and regulatory compliance domain by developing advanced\n                                generative AI solutions, including a horizon scanning tool, an obligations generation\n                                tool, and an LLM-based expert system. Our approach combines the LangChain\n                                framework, OpenAI’s GPT-4, text embeddings, and prompt engineering techniques\n                                to effectively reduce hallucinations and generate reliable and accurate domain-\n                                specific outputs. A human-in-the-loop control mechanism is used as a final backstop\n                                to ensure accuracy and mitigate risk. Our findings emphasise the role of LLMs as\n                                foundation engines in specialist tools and lay the groundwork for building the next\n            

In [46]:
len(texts)

13

In [47]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [48]:
docs

[Document(page_content='Gracenote.ai: Legal Generative AI for Regulatory Compliance\nJules Ioannidis, Joshua Harper, Ming Sheng Quah 1 and Dan Hunter 1, 2\n1\n    Gracenote.ai, Melbourne Australia\n2\n    The Dickson Poon School of Law, King’s College London, United Kingdom*', metadata={}),
 Document(page_content='Abstract: We investigate the transformative potential of large language models\n                                (LLMs) in the legal and regulatory compliance domain by developing advanced\n                                generative AI solutions, including a horizon scanning tool, an obligations generation\n                                tool, and an LLM-based expert system. Our approach combines the LangChain\n                                framework, OpenAI’s GPT-4, text embeddings, and prompt engineering techniques\n                                to effectively reduce hallucinations and generate reliable and accurate domain-\n                                specific outp

##  3 types of CombineDocuments Chains

[Taken from the LangChain Docs](https://langchain.readthedocs.io/en/latest/modules/indexes/combine_docs.html)

## Summarize Simple with map_reduce

### Map Reduce
This method involves **an initial prompt on each chunk of data ***
( for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). **Then a different prompt is run to combine all the initial outputs.** This is implemented in the LangChain as the MapReduceDocumentsChain.

**Pros:** Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. Loses some information during the final combining call.

In [49]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [50]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  This article explores the potential of large language models (LLMs) to create advanced generative
AI solutions for legal and regulatory compliance. LLMs have been used to explore natural language
parsing, pass legal tests, and even replace lawyers. Technologies such as AutoGPT, GPT-4, LangChain,
text embeddings, and prompt engineering are used to reduce hallucinations and generate reliable
outputs. Future research focuses on extending support across multiple jurisdictions and languages,
refining prompts, and developing autonomous AI agents.


In [51]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [52]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [53]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Gracenote.ai: Legal Generative AI for Regulatory Compliance
Jules Ioannidis, Joshua Harper, Ming Sheng Quah 1 and Dan Hunter 1, 2
1
    Gracenote.ai, Melbourne Australia
2
    The Dickson Poon School of Law, King’s College London, United Kingdom*"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"Abstract: We investigate the transformative potential of large language models
                                (LLMs) in the legal and regulatory compliance domain by developing advanced
                                generative AI solutions, including a horizon scanning tool, an obligations generation
                                tool, and an LLM-based expert system. Our approach combines the LangChain
                                framework, OpenAI’s GPT-4, text embeddings, and prompt engineerin

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


> Finished chain.

> Finished chain.

> Finished chain.
 This article discusses the use of AI in legal compliance, specifically focusing on AutoGPT, GPT-4,
LangChain, large language models, legal generative AI, legal text embeddings, prompt engineering,
and regulation. It investigates the potential of large language models for legal and regulatory
compliance, developing AI solutions such as a horizon scanning tool, an obligations generation tool,
and an LLM-based expert system. It emphasizes LLMs as essential engines for specialist tools and
lays the groundwork for building the next generation of legal and compliance applications.


### Summarizing with the 'stuff' Chain



### Stuffing
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

**Pros:** Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.

**Cons:** Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

The main downside of this method is that **it only works one smaller pieces of data.**  Once you are working with many pieces of data, this approach is no longer feasible. The next two approaches are designed to help deal with that.



In [54]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [55]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])


In [56]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 
- Gracenote.ai is a legal generative AI for regulatory compliance developed by Jules Ioannidis,
Joshua Harper, Ming Sheng Quah and Dan Hunter
- Combines LangChain framework, OpenAI’s GPT-4, text
embeddings and prompt engineering techniques to reduce hallucinations and generate accurate outputs
- Human-in-the-loop control mechanism used as a backstop to ensure accuracy and mitigate risk
- LLMs
used as foundation engines in specialist tools
- Future research will focus on extending support
across multiple jurisdictions and languages, refining prompts and text embedding datasets, and
developing autonomous AI agents and robust LLM-based expert systems.


### Ver 3 With 'map_reduce' with our custom prompt

In [57]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=BULLET_POINT_PROMPT,
                             combine_prompt=BULLET_POINT_PROMPT)

# chain.llm_chain.prompt= BULLET_POINT_PROMPT
# chain.combine_document_chain.llm_chain.prompt= BULLET_POINT_PROMPT

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


- Gracenote.ai is a legal generative AI for regulatory compliance developed by Jules Ioannidis,
Joshua Harper, Ming Sheng Quah and Dan Hunter, based in Melbourne, Australia and The Dickson Poon
School of Law, King’s College London, United Kingdom
- Uses LLMs, text embeddings, and prompt
engineering techniques with a human-in-the-loop control mechanism to ensure accuracy and mitigate
risk
- AutoGPT and LangChain are two examples of legal generative AI used to create legal documents
- Prompt engineering and fine-tuning are two standard approaches used to generate legal text


In [58]:
# with a custom prompt
prompt_template = """Write a concise summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

## with intermediate steps
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="map_reduce",
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco


• Gracenote.ai is a legal generative AI for regulatory compliance developed by a team based in
Melbourne, Australia and The Dickson Poon School of Law, King’s College London, United Kingdom. 
•
AutoGPT is a legal generative AI that uses large language models, expert systems, and prompt
engineering to generate legal text embeddings.
• GPT-4 and LangChain are two of the most popular GRC
(Governance, Risk, and Compliance) tools used to help organizations comply with regulations.
• LLMs
(Large Language Models) are often referred to as “foundation models” and can be used to drive a
range of products and services.
• Papers have been written exploring the use of LLMs in law,
including whether GPT can pass the bar exam or other legal tests, as well as papers predicting the
death of lawyers by LLMs.


In [59]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][0],
                             width=10,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



•
Gracenote.ai
is a legal
generative
AI for
regulatory
compliance
developed
by Jules
Ioannidis,
Joshua
Harper,
Ming Sheng
Quah, and
Dan
Hunter. 
•
The team
is based
in
Melbourne,
Australia
and The
Dickson
Poon
School of
Law,
King’s
College
London,
United
Kingdom.
•
Gracenote.ai
is
designed
to help
lawyers
and legal
professionals
automate
the
process of
creating
legal
documents,
saving
time and
money. 
•
The AI is
able to
generate
legal
documents
that are
tailored
to the
specific
needs of
the user,
ensuring
accuracy
and
compliance
with
regulations.


### With the 'refine' CombineDocument Chain

## Refine
This method involves **an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document**, asking the LLM to refine the output based on the new document.

**Pros:** Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

In [60]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

  Gracenote.ai is a legal generative AI for regulatory compliance developed by Jules Ioannidis,
Joshua Harper, Ming Sheng Quah, and Dan Hunter from Gracenote.ai in Melbourne, Australia, and the
Dickson Poon School of Law at King’s College London in the United Kingdom. The AI combines the
LangChain framework, OpenAI’s GPT-4, text embeddings, AutoGPT, GRC, large language models (LLMs)
often called “foundation models”, and prompt engineering techniques to effectively reduce
hallucinations and generate reliable and accurate domain-specific outputs. It also features a human-
in-the-loop control mechanism to ensure accuracy and mitigate risk. This AI lays the groundwork for
building the next generation of legal and compliance applications, with future research focused on
extending support across multiple jurisdictions and languages, refining prompts and text embedding
datasets, and developing autonomous AI agents and robust LLM-based expert systems.


In [61]:
prompt_template = """Write a concise summary of the following extracting the key information:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=PROMPT,
                             refine_prompt=refine_prompt)


In [62]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xDjxK3O2naZzvtFMMChWhYwt on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco



Gracenote.ai is a legal generative AI developed by Jules Ioannidis, Joshua Harper, Ming Sheng Quah
and Dan Hunter from Gracenote.ai in Melbourne, Australia and The Dickson Poon School of Law at
King's College London, United Kingdom. It combines the LangChain framework, OpenAI’s GPT-4, text
embeddings, and prompt engineering techniques to effectively reduce hallucinations and generate
reliable and accurate domain-specific outputs. The AutoGPT algorithm is used to generate legal text
embeddings for GRC (Governance, Risk and Compliance) and large language models (LLMs) are used as a
foundation to drive a range of products and services. This is done through prompt engineering and
fine-tuning, which involves adapting a pre-trained LLM to a specific task or domain by training it
on a specialised dataset. A human-in-the-loop control mechanism is used as a final backstop to
ensure accuracy and mitigate risk, and future research will focus on extending support across
multiple jurisdictions an

In [63]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][0],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

 Gracenote.ai is a legal generative AI developed by Jules Ioannidis, Joshua Harper, Ming Sheng Quah
and Dan Hunter from Gracenote.ai in Melbourne, Australia and The Dickson Poon School of Law at
King's College London, United Kingdom, designed to help with regulatory compliance.
